## Stable Diffusion 2 Text-to-Image Generation on IPU

This notebook demonstrates how a stable diffusion 2 inference pipeline can be run on Graphcore IPUs.

![Text to image with SD2](sample_images/text_to_image_sd2.png)

### Requirements

* An enabled Poplar SDK environment (or Paperspace account with access to the PyTorch IPU runtime)
* Additional dependencies installable via pip (done below)
* Access to the pretrained Stable-Diffusion-v2 checkpoint (done below)

In [ ]:
%%capture
!pip install -r requirements.txt
!pip install "ipywidgets>=7,<8"

Values for the virtual IPU Pod size and the cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod16")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/stablediffusion2_text2img"

To download the pretrained Stable-Diffusion-v2 checkpoint, we must first authenticate to the Hugging Face Hub. Begin by creating a read access token on the [Hugging Face website](https://huggingface.co/settings/tokens) (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your read token:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

If you have not done so already, you will need to accept the User License on the [model page](https://huggingface.co/stabilityai/stable-diffusion-2).

### Pipeline creation

We are now ready to import and run the pipeline.

In [ ]:
import torch
from diffusers import DPMSolverMultistepScheduler

from ipu_models import IPUStableDiffusionPipeline

In [ ]:
pipe = IPUStableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2",
    revision="fp16", 
    torch_dtype=torch.float16,
    ipu_config={
        "matmul_proportion": [0.06, 0.1, 0.1, 0.1],
        "executable_cache_dir": executable_cache_dir,
    },
    requires_safety_checker=False
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_attention_slicing()

In [ ]:
image_width = os.getenv("STABLE_DIFFUSION_2_TXT2IMG_DEFAULT_WIDTH", default=768)
image_height = os.getenv("STABLE_DIFFUSION_2_TXT2IMG_DEFAULT_HEIGHT", default=768)

We run a dummy generation step to trigger the one-time compilation process. This should take on the order of 20 minutes.

In [ ]:
pipe("apple", height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9);

### Image generation

Below you will find some example prompts. We encourage you to try your own!

In [ ]:
prompt = "a shiba inu in a zen garden, acrylic painting"
pipe(prompt, height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9).images[0]

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
pipe(prompt, height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9).images[0]

In [ ]:
prompt = "the living room of a cozy wooden house with a fireplace"
out = pipe(prompt, height=image_height, width=image_width, num_inference_steps=25, guidance_scale=9)
out.images[0]

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1,1)
fig.set_size_inches(9, 9)
ax.imshow(out.images[0])

ax.set_title(f"Prompt: {prompt}")
ax.axis("off")
fig.savefig("sample_images/text_to_image_sd2.png", dpi=150)